In [1]:
import json
import faiss
import numpy as np
import openai

# Setting up my OpenAI client
client = openai.OpenAI(
    api_key="Your Open API Key"
)

# Function to split text into smaller chunks for processing
def chunk_text(text, max_tokens=1500):
    words = text.split()
    for i in range(0, len(words), max_tokens):
        yield " ".join(words[i:i + max_tokens])

# Reading my ACT article so I can use it as context
with open("article.txt", "r", encoding="utf-8", errors="ignore") as f:
    full_text = f.read()

# Splitting the ACT article into smaller sections so embeddings are easier
def split_text(text, max_tokens=500):
    paragraphs = text.split("\n")
    chunks, current = [], ""
    token_estimate = lambda x: len(x.split())
    for p in paragraphs:
        p = p.strip()
        if not p:
            continue
        if token_estimate(current + p) < max_tokens:
            current += " " + p
        else:
            chunks.append(current.strip())
            current = p
    if current:
        chunks.append(current.strip())
    return chunks

# Creating the article chunks
act_chunks = split_text(full_text)
print(f"ACT article split into {len(act_chunks)} chunks.")

# Getting embeddings for each chunk so we can use FAISS for semantic search
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-small",  
        input=[text]
    )
    return response.data[0].embedding

embeddings = [get_embedding(chunk) for chunk in act_chunks]
print("Embedded all ACT article chunks.")

# Building FAISS index for retrieving relevant ACT theory sections
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))

# Loading the therapy session transcript
with open("Session Two.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_text = f.read()

# Splitting the therapy session into manageable parts
session_chunks = list(chunk_text(session_text, max_tokens=1500))
print(f"Session split into {len(session_chunks)} chunks.")

# Summarizing each chunk using ACT theory as context
chunk_summaries = []
for i, chunk in enumerate(session_chunks, 1):
    query_embedding = get_embedding(chunk)
    k = 5
    D, I = index.search(np.array([query_embedding]).astype("float32"), k)
    retrieved = [act_chunks[j] for j in I[0]]

    context = "\n\n".join(retrieved)
    prompt = f"""You are an expert ACT therapist.

Using the following ACT theory context:
---
{context}
---

Summarize this part of a therapy session by highlighting ACT principles used or reflected:
---
{chunk}
---
"""

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "You are a clinical psychologist summarizing therapy sessions using ACT theory."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
        max_tokens=600
    )

    summary = response.choices[0].message.content.strip()
    chunk_summaries.append(summary)
    print(f"Summarized chunk {i}/{len(session_chunks)}")

# Combining all partial summaries into one final ACT-informed summary
final_prompt = "Combine these partial summaries into one cohesive ACT-informed summary:\n\n" + "\n\n".join(chunk_summaries)

final_response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": "You are a clinical psychologist combining session summaries into a final ACT-informed summary."},
        {"role": "user", "content": final_prompt}
    ],
    temperature=0.4,
    max_tokens=800
)

final_summary = final_response.choices[0].message.content.strip()

print("\nACT-Informed Final Summary:\n")
print(final_summary)


ACT article split into 12 chunks.
Embedded all ACT article chunks.
Session split into 5 chunks.
Summarized chunk 1/5
Summarized chunk 2/5
Summarized chunk 3/5
Summarized chunk 4/5
Summarized chunk 5/5

=== ACT-Informed Final Summary ===

Certainly! Here is a cohesive, ACT-informed summary that synthesizes the key themes and processes reflected across the session segments you provided:

---

**ACT-Informed Session Summary**

Throughout these therapy sessions, the therapist skillfully employs core Acceptance and Commitment Therapy (ACT) processes to foster the client’s psychological flexibility and movement toward a more values-driven life. Several ACT principles are woven throughout the client’s experiences and the therapist’s interventions:

---

### 1. **Cognitive Defusion**
The client increasingly notices and describes their thoughts and feelings as transient mental events (“a brief flare of anger,” “the back of my mind is working”), rather than absolute truths or defining features o